In [1]:
import image_similarity_measures
#from image_similarity_measures.quality_metrics import rmse, psnr
from image_similarity_measures.quality_metrics import metric_functions
import cv2
import pandas as pd
import numpy as np
from typing import List
import time
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn_image as isns
from PIL import Image, ImageOps
import glob
#import PIL


In [2]:
#VAR declarations
all_metrics =  ['ssim']
# maximum umber of frames to capture
max_frames = 150;

#comparison_images = [Image.open(img) for img in glob.glob("img/*.jpg")]
#image_labels = [img.split("/",1)[-1] for img in glob.glob("img/*.jpg")]
#comparison_images = [img for img in glob.glob("img/*.jpg")]

In [3]:
cap = cv2.VideoCapture('testvid.mov')
#Or most common ID for webcams:
#cap = cv2.VideoCapture(0)

In [6]:
fps = cap.get(cv2.CAP_PROP_FPS)
print(f"Frames per second using video.get(cv2.CAP_PROP_FPS) : {fps}")

Frames per second using video.get(cv2.CAP_PROP_FPS) : 30.029060381013885


In [ ]:
frame_nr = 0
while cap.isOpened():
    ret, frame = cap.read()

    if ret:
        print(f"frame{frame_nr}")
        #cv2.imwrite(f"frame{frame_nr}.jpg", frame)
        #i.e. at 30 fps, this advances one second
        #int(fps) for rounding
        frame_nr += int(fps)  
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_nr)
    else:
        cap.release()
        break

In [ ]:

#cap.release()

In [ ]:
def compare(df: pd.DataFrame, path_a: str, path_b: str, metrics: List[str]) -> pd.DataFrame:
    img_a = cv2.imread(path_a)
    img_b = cv2.imread(path_b)

    for metric in metrics:
        metric_func = metric_functions[metric]
        start_time = time.time()
        out_value = float(metric_func(img_a, img_b))
        end_time = time.time()
        output_dict = {}
        output_dict['metric'] = metric
        output_dict['x'] = path_a
        output_dict['y'] = path_b
        output_dict['value'] = out_value
        output_dict['walltime'] = end_time - start_time
        df = df.append(output_dict,ignore_index=True)
        output_dict['x'] = path_b
        output_dict['y'] = path_a
        df = df.append(output_dict,ignore_index=True)
        
    return df


In [ ]:
df = pd.DataFrame()
for idx_x, x in enumerate(comparison_images):
    for idx_y, y in enumerate(comparison_images):
        # remove duplicate comparisons: (x,y) == (y,x) or (x,x)
        if idx_x >= idx_y:
            continue
        df = compare(df, x, y, all_metrics)
#df
